In [1]:
import pandas as pd
import numpy as np

import scipy.io as sio
from scipy.linalg import norm
from sklearn.preprocessing import scale, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
def loss_func(X,y_onehot,W):
    z = np.dot(X, W)
    h = sigmoid(z)
    
    loss = np.sum(np.multiply(-y_onehot, np.log(h+0.0001)) + np.multiply(-(1-y_onehot), np.log(1-h +0.0001))) / len(X) 
    
    return loss

In [4]:
# 从数据集中读取全部数据并且分割为训练集和测试集
def readData(test_size):
    domains = ['amazon', 'caltech', 'dslr', 'webcam']
    
    X_all = np.empty((0, 4096))
    y_all = np.empty((0),dtype=np.int)
    
    for domain in domains:
        data = sio.loadmat('OC10_vgg6/' + domain + '.mat')
        # Xs 为Vgg抽取的特征，ys为对应的class
        Xs, ys = data['FTS'].astype(np.float64), data['LABELS'].ravel()
        # ys从0开始？
        ys = LabelEncoder().fit(ys).transform(ys).astype(np.int)
        # Xs 标准化
        Xs = Xs / norm(Xs, axis=1, keepdims=True)
        
        # 不同数据集拼接到一起
        X_all = np.concatenate((X_all,Xs))
        y_all = np.concatenate((y_all,ys))
        
    return train_test_split(X_all, y_all, test_size=test_size)

In [5]:
# 声明训练参数
# 训练集测试集划分比例
test_size = 0.3

# 学习率
learning_rate=0.01

# 最大迭代次数
maxt = 1000

#正太分布标准差
std = 1e-3

max_loss = 0.001


In [6]:
# 按比例读取训练集和测试集
X,X_test,y,y_test = readData(test_size)

n_classes = len(np.unique(y)) # class数量
n_samples = X.shape[0]  # 样本数量
n_features = X.shape[1]  # 特征维度

W = np.random.uniform(0,1,size=(n_features, n_classes))
b = np.zeros(n_classes)

y_onehot = np.eye(10)[y]


In [7]:
for _ in range(maxt):
    scores = np.dot(X, W)
    exp_scores = sigmoid(scores)
    
    dW = np.dot(X.T, exp_scores- y_onehot ) 
    W-=learning_rate * dW
    
    print(loss_func(X,y_onehot,W))


8.937129778768137
8.54037946792519
8.215831960785843
7.946468094319802
7.718862132024909
7.522947534604076
7.351385875868105
7.198877418851154
7.06158039430423
6.936676419712062
6.822063175460318
6.716142499682639
6.61767545410565
6.52568300353187
6.439377404727378
6.358114246267623
6.281358446066014
6.208659767813732
6.139634904253012
6.073954150654393
6.01133133263846
5.951516075508223
5.894287783627754
5.839450887263805
5.786831042442308
5.73627205730383
5.687633379528197
5.640788022377364
5.5956208375377425
5.552027065058304
5.509911106849857
5.46918548217827
5.42976993254599
5.391590650142162
5.354579609234296
5.318673983887292
5.283815638525464
5.24995068031551
5.217029064301087
5.185004243779924
5.1538328596696585
5.123474463625299
5.093891270499376
5.065047936414592
5.036911359277875
5.009450499027734
4.982636215291996
4.956441120455155
4.930839446404934
4.905806923455929
4.8813206701414
4.857359092728781
4.833901793454904
4.810929486597438
4.788423921602708
4.766367812579755
4

In [8]:
y_test_socre  = np.dot(X_test,W)
y_test_exp_scores = np.exp(y_test_socre)
y_test_probs = y_test_exp_scores / np.sum(y_test_exp_scores, axis=1, keepdims=True)
y_test_probs_max = np.argmax(y_test_probs, axis=1)
print(classification_report(y_test, y_test_probs_max))

              precision    recall  f1-score   support

           0       0.94      0.93      0.93        94
           1       1.00      0.99      0.99        69
           2       0.89      0.88      0.88        65
           3       0.97      0.91      0.94        93
           4       0.82      0.83      0.82        75
           5       0.87      0.87      0.87        85
           6       0.89      0.91      0.90        80
           7       0.84      0.94      0.89        63
           8       0.97      0.94      0.95        64
           9       0.92      0.92      0.92        72

    accuracy                           0.91       760
   macro avg       0.91      0.91      0.91       760
weighted avg       0.91      0.91      0.91       760

